# Generate Reports


## Subject wise Report

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import openpyxl
import re
import os
from variableUtils import *
from Utils import *
from ClassUtils import *
from pprint import pprint
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter, landscape, A4, A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Spacer, Image
from reportlab.lib import colors
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.platypus import Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from PyPDF2 import PdfReader, PdfWriter
from io import BytesIO

from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

(841.68, 1190.8799999999999)


In [2]:
workbookPath = 'FullSpreadsheets\CAF+v0.1_August+6,+2024\CAF v0.1_August 6, 2024_00.50.xlsx'
savePath =  'FullSpreadsheets\CAF+v0.1_August+6,+2024\CAF v0.1_August 6, 2024_00.50 filtered.xlsx'
# guttmanPath = 'FullSpreadsheets\CAF+v0.1_August+6,+2024\CAF v0.1_August 6, 2024_00.50 guttman.xlsx'
guttmanPath = 'FullSpreadsheets\CAF+v0.1_August+18,+2024\CAF v0.1_August 18, 2024_19.04 guttman.xlsx'
guttmanPath = 'DDS2\CAF+v0.1_September+2,+2024_22.43\CAF v0.1_September 2, 2024_22.43 endo guttman.xlsx'
guttmanPath = 'FullSpreadsheets\CAF+v0.1_September+12,+2024_21.55\CAF v0.1_September 12 August guttman.xlsx'
guttmanPath = 'FullSpreadsheets\CAF+v0.1_December+5,+2024\CAF v0.1_December 5, 2024 guttman.xlsx'
# guttmanPath = 'BOH2\CAF+v0.1_October152024_19.01\CAF v0.1_October 15, 2024_19.01 Paeds guttman.xlsx'
workbook, folder, file = loadWorkbook(guttmanPath)


('Loaded workbook: FullSpreadsheets\\CAF+v0.1_December+5,+2024 | CAF '
 'v0.1_December 5, 2024 guttman | .xlsx')
("Workbook sheets: ['011_COE', '012_POE', '013_LIMITED OE', "
 "'014_CONSULTATION', '022_I-O RAD', '061_VITALITY', '071_DIAGNOSTIC MODEL', "
 "'072_PHOTOS', '074_PHYSICAL MODELS', '111_CLINIC', '111_SIM', '113_CLINIC', "
 "'113_SIM', '114_CLINIC', '114_SIM', '115_CLINIC', '121_CLINIC REMIN', "
 "'121_SIM REMIN', '123_CLINIC', '123_SIM', '131_CLINIC', '141_CLINIC', "
 "'142_CLINIC', '161_CLINIC', '161_SIM', '165_CLINIC', '165_SIM', "
 "'221_CLINIC', '221_SIM', '222_CLINIC', '222_SIM', '311_CLINIC', '311_SIM', "
 "'386_CLINIC', '386_SIM', '411_CLINIC', '411_SIM', '414_CLINIC CVEK', "
 "'414_CLINIC DECIDUOUS', '414_SIM CVEK', '414_SIM DECIDUOUS', '415_SIM "
 "ACCESS', '415_SIM WORK LENGTH', '416_SIM RCT ADD', '417_SIM CONE FIT', "
 "'417_SIM ROOT FLG', '418_SIM CONE FIT', '418_SIM ROOT FLG', '511_CLINIC', "
 "'511_SIM', '512_CLINIC', '513_SIM', '514_SIM', '521_CLINIC', '521_SIM

In [ ]:
# path1 = 'DDS2\CAF+v0.1_September+2,+2024_22.43\CAF v0.1_September 2, 2024_22.43 endo filtered.xlsx'
# path2 = 'DDS2\CAF+v0.1_September+2,+2024_22.43\CAF v0.1_September 2, 2024_22.43 endo guttman marks.xlsx'
# checkAttendence(path2, 'DDS2 (2024)')
# df = pd.read_excel(path1)
# studentsAttented = df['Student ID'].unique()

In [3]:
# Create a dictionary with Subjects and the item numbers they have
subjectDict= {}
for i, sheet in enumerate(workbook.sheetnames):
    # print(sheet)
    code = sheet.split('_')[0]
    df = loadDfFromSheet(workbook, sheet)
    # df = df[~df[colId].isin([111111, 1111111, 1, 12, 123, 1234, 12345, 123456, 1234567, 12345678])]
    sub = df[colSubject].unique()
    # print(sub)
    # remove None
    sub = [s for s in sub if s]
    for s in sub:
        if s not in subjectDict.keys():
            subjectDict[s] = {}
        subdf = df[df[colSubject] == s]
        if len(subdf) > 0:
            subjectDict[s][sheet] = subdf
        
    # if i >= 4:
    #     break



# Do for DENT90115

In [ ]:
dent90115Dict = {}
for i, sheet in enumerate(workbook.sheetnames):
    code = sheet.split(' ')[0]
    df = loadDfFromSheet(workbook, sheet)
    dent90115Dict[sheet] = df

## Paeds

In [ ]:
labelBlank = 'Not Filled'
otherCols = [colComplex, colClinicType, colDate, colSupervisor, colFinished, colCE, colCEReason]
labelRubric = 'Rubric Weighted Score'
# rubricQues.remove('TS')
dfTemplate = pd.DataFrame(columns=[colId, 'Yes', 'No', 'Not Reviewed', labelBlank, 'Total MC items'] + rubricQues + ['MC Score', labelRubric, 'ES Weighted Score', 'Total Score'] + otherCols +['Item'])
notReviewedW = 0.5
rubricW = {'PS': 0.05, 'CS': 0.05, 'TS': .1, 'ES': .1} # This is simulation thus professionalism and communication are reduced weighted
rubricDenom = {'PS': 2, 'CS': 4, 'TS': 4, 'ES': 4}
mcScoreW = 0.8
rubricScoreW = 0.2
# esW = 0.05
  

In [ ]:
# path = 'BOH3\\25-08-2004\BOH3 Viva Voce Assessment 2024_August 25, 2024_03.48 guttman.xlsx'
# df_ = pd.read_excel(path, keep_default_na=False)
# folder, filename, ext = getFolderandFileName(path)
# dont' take last row
df_ = df_.iloc[:-1]
# for i, col in enumerate(df_.columns):
#     print(i, col)
dent90115Dict = {'414': df_}
key0 = list(dent90115Dict.keys())[0]
dfPaed = dfTemplate.copy()  

rubricQuesNoES = rubricQues.copy()

for key, df in dent90115Dict.items():
    print(key)
    code = key.split('_')[0]
    df[colId] = df[colId].astype('Int64', errors='ignore')
    df = df[df[colFinished] == 'True']

    mcColumns = findMCColumns(df, code)
    mcColumns = [col for col in df.columns if '#' in col]
    print(mcColumns)
    print(df[colId].value_counts())
    aggFuncs = {col: 'first' for col in df.columns if col not in mcColumns + rubricQues}
    aggFuncs.update({col: custom_agg for col in mcColumns})
    aggFuncs.update({col: 'max' for col in rubricQues})
    df = df.groupby([colId], as_index=False).agg(aggFuncs)  
    print(df[colId].value_counts()) 
    display(df[mcColumns].head())

    pairs_df = getPairCounts(df)
    pairs_df.to_excel(f'{folder}/ pairs.xlsx', index=False)
    countsDf = dfTemplate.copy()
    countsDf[colId] = df[colId]
    countsDf['Yes'] = df[mcColumns].apply(lambda x: (x == 1).sum() + (x=='1').sum(), axis=1)
    countsDf['No'] = df[mcColumns].apply(lambda x: (x == 0).sum() + (x=='0').sum(), axis=1)
    countsDf['Not Reviewed'] = df[mcColumns].apply(lambda x: (x == 'NA').sum(), axis=1)
    countsDf[labelBlank] = df[mcColumns].apply(lambda x: (x == '').sum() + x.isna().sum() + (x=='None').sum(), axis=1)
    countsDf['Total MC items'] = len(mcColumns)
    
    for q in rubricQues:
        countsDf[q] = df[q]
    for col in otherCols:
        if col in df.columns:
            countsDf[col] = df[col]

    countsDf['MC Score'] = (countsDf['Yes'] + notReviewedW*countsDf['Not Reviewed']) / (countsDf['Yes'] + countsDf['No'] + notReviewedW*countsDf['Not Reviewed'])
    countsDf['MC Score'] = countsDf['MC Score'].apply(lambda x: round(x, 2))
    
    display(countsDf.head())
    
    # display(countsDf[rubricQues])
    countsDf[labelRubric] = countsDf.apply(lambda row: sum(row[ques] * rubricW[ques] / rubricDenom[ques] for ques in rubricQuesNoES), axis=1)/0.3
    countsDf[labelRubric] = countsDf[labelRubric].apply(lambda x: round(x, 2))
    
    countsDf['ES Weighted Score'] = countsDf['ES']/ rubricDenom['ES']
    countsDf['ES Weighted Score'] = countsDf['ES Weighted Score'].apply(lambda x: round(x, 2))
    
    countsDf['Total Score'] = (mcScoreW*countsDf['MC Score'] + rubricScoreW*countsDf[labelRubric])*100
    countsDf['Total Score'] = countsDf['Total Score'].apply(lambda x: round(x, 2))

    # Add 20% penalty if critical error is yes
    countsDf['Total Score CE Penalty (20%)'] = countsDf.apply(lambda row: row['Total Score'] * 0.8 if row[colCE] == 'Yes' else row['Total Score'], axis=1)
    countsDf['Total Score CE Penalty (20%)'] = countsDf['Total Score CE Penalty (20%)'].apply(lambda x: round(x, 2))

    # Add 10% penalty if critical error is yes
    countsDf['Total Score CE Penalty (10%)'] = countsDf.apply(lambda row: row['Total Score'] * 0.90 if row[colCE] == 'Yes' else row['Total Score'], axis=1)
    countsDf['Total Score CE Penalty (10%)'] = countsDf['Total Score CE Penalty (10%)'].apply(lambda x: round(x, 2))

    # countsDf['Total Score'] = (mcScoreW*countsDf['MC Score'] + rubricScoreW*countsDf['Rubric Weighted Score'])*100
    # countsDf['Total Score'] = countsDf['Total Score'].apply(lambda x: round(x, 2))
    # get correlation matrix between MC and Rubric bar ES and ES
    countsDf['Item'] = key
    # countsDf = countsDf[countsDf[colClinicType] == 'Paeds']
    # display(countsDf.head())
    countsDfPaed = pd.concat([countsDfPaed, countsDf])
    average_scores = df.groupby('CE Name')['Total Score'].mean().reset_index()
    # get counts of each CE Name
    counts = df['CE Name'].value_counts().reset_index()
    counts.columns = ['CE Name', 'Count']
    display(counts)

    # plot a bar chart of average scores
    plt.figure(figsize=(12, 6))
    sns.barplot(x='CE Name', y='Total Score', data=average_scores)
    plt.title(f'Average Scores for examiners')
    plt.xlabel('Examiner')
    plt.xticks(rotation=90)
    plt.show()
    # for i, row in df.iterrows():
    #     if row[colClinicType] == 'Paeds':
    #         print(f'Paeds row found: {row[colId]} {row[colDate]} {row[colSupervisor]}')

In [ ]:
display(countsDfPaed.head())
correlation_matrix = countsDfPaed[['MC Score', labelRubric, 'ES Weighted Score']].corr()
display(correlation_matrix)
correlation_matrix.to_csv(f'{folder}\\correlation_matrix.csv')
print(countsDfPaed[colId].value_counts())

In [ ]:
# Get distribution of total scores
fig, ax = plt.subplots()
sns.histplot(countsDfPaed['Total Score'], bins=range(0, 105, 5), ax=ax)
ax.set_title('Distribution of Total Scores')
ax.set_xlabel('Total Score')
ax.set_xticks(range(0, 105, 5))

# ES distribution
fig, ax = plt.subplots()
sns.histplot(countsDfPaed['ES'], bins=[0,1,2,3,4], ax=ax)
ax.set_title('Distribution of ES Scores')
ax.set_xlabel('ES Weighted Score')
ax.set_xticks([0,1,2,3,4])

# MC distribution
fig, ax = plt.subplots()
sns.histplot(countsDfPaed['MC Score'], bins=np.arange(0, 1.1, 0.1), ax=ax)
ax.set_title('Distribution of MC Scores')
ax.set_xlabel('MC Score')
ax.set_xticks(np.arange(0, 1.1, 0.1))


In [ ]:
countsDfPaed.to_csv(f'{folder}/{filename} marks.csv', index=False)

## Endo and Fixed Pros

### Endo Guttman


In [ ]:
notReviewedW = 0.5
labelBlank = 'Not Filled'
labelRubric = 'Rubric Score'
rubricW = {'PS': 0.05, 'CS': 0.05, 'TS': .1, 'ES': .1} # This is simulation thus professionalism and communication are reduced weighted
rubricDenom = {'PS': 2, 'CS': 2, 'TS': 4, 'ES': 4}
mcScoreW = 0.8
rubricScoreW = 0.2
rubricQues = ['PS', 'CS', 'TS', 'ES']

for key, df in dent90115Dict.items():
    code = key.split('_')[0]
    if not code.startswith('4'):
        continue
    print(key)
    # Filter out the dates as 14th of August
    # df[colDate] = pd.to_datetime(df[colDate])
    # df = df[df[colDate].dt.day == 14] # 14th of August
    # df = df[df[colDate].dt.month == 8]   # August
    # df[colDate] = df[colDate].dt.strftime('%d-%m-%Y')

    mcColumns = findMCColumns(df, code)
    df[colCEReason] = df[colCEReason].fillna(' ')
    df[colCEReason] = df[colCEReason].astype(str)
    display(df.head())
    df = aggregator(df, mcColumns, colCE, colCEReason)
    display(df.head())
    # remove these columns ICC_MC1_supervisor	ICC_MC2_supervisor	RKC_MC1_supervisor	CC_MC1_supervisor Patient	Complexity	Patient Age	Role Diagnostics	OMFS Sim/Clinic	General services	Rem Pros CLINIC	Paeds specific	Preventive, Prophylactic and Bleaching Services	Periodontics	Endodontics	Restorative Services	Fixed Prosthodontics Weighted	Weighted_penalty_10	Weighted_penalty_20
    df = df.drop(columns=['ICC_MC1_supervisor', 'ICC_MC2_supervisor', 'RKC_MC1_supervisor', 'CC_MC1_supervisor', 'Patient', 'Complexity', 'Patient Age', 'Role', 'Diagnostics', 'OMFS Sim/Clinic', 'General services', 'Rem Pros CLINIC', 'Paeds specific', 'Preventive, Prophylactic and Bleaching Services', 'Periodontics', 'Endodontics',
                           'Restorative Services', 'Fixed Prosthodontics', 'Weighted', 'Weighted_penalty_10', 'Weighted_penalty_20'], errors='ignore')
    



    # Create Guttmann for each key
    if len(df) == 0:
        continue
    df['Yes'] = df[mcColumns].apply(lambda x: (x == 1).sum() + (x=='1').sum(), axis=1)
    df['No'] = df[mcColumns].apply(lambda x: (x == 0).sum() + (x=='0').sum(), axis=1)
    df['Not Reviewed'] = df[mcColumns].apply(lambda x: (x == 'NA').sum(), axis=1)
    df[labelBlank] = df[mcColumns].apply(lambda x: (x == '').sum() + x.isna().sum() + (x=='None').sum(), axis=1)
    
    df['MC Score'] = (df['Yes'] + notReviewedW*df['Not Reviewed']) / (df['Yes'] + df['No'] + notReviewedW*df['Not Reviewed'])
    df['MC Score'] = df['MC Score'].apply(lambda x: round(x, 2))
    
    df[labelRubric] = df.apply(lambda row: sum(row[ques] * rubricW[ques] / rubricDenom[ques] for ques in rubricQues), axis=1)/0.3
    df[labelRubric] = df[labelRubric].apply(lambda x: round(x, 2))
    
    df['Total Score'] = (mcScoreW*df['MC Score'] + rubricScoreW*df[labelRubric])*100
    df['Total Score'] = df['Total Score'].apply(lambda x: round(x, 2))
    df.sort_values('Total Score', ascending=False, inplace=True)    
    # display(df.head())
    saveDf(df, f'{folder}/{file} Endo.xlsx', key.replace('/', '-'), len(mcColumns) + 12)

In [ ]:
labelBlank = 'Not Filled'
otherCols = [colComplex, colDate, colFinished, colCE, colCEReason, colClinicType, colComments]
labelRubric = 'Rubric Weighted Score'
dfTemplate = pd.DataFrame(columns=[colId, 'Yes', 'No', 'Not Reviewed', labelBlank, 'Total MC items'] + rubricQues + ['MC Score', labelRubric, 'ES Weighted Score', 'Total Score', 
                                'Total Score CE Penalty (20%)', 'Total Score CE Penalty (10%)'] + otherCols +['Item'])
notReviewedW = 0.5
rubricW = {'PS': 0.05, 'CS': 0.05, 'TS': .1, 'ES': .1} # This is simulation thus professionalism and communication are reduced weighted
rubricDenom = {'PS': 2, 'CS': 2, 'TS': 4, 'ES': 4}
mcScoreW = 0.8
rubricScoreW = 0.20
# esW = 0.05

In [ ]:
key0 = list(dent90115Dict.keys())[0]
countsDfPaed = dfTemplate.copy()  
rubricQuesNoES = rubricQues.copy()
# rubricQuesNoES.remove('ES')
for key, df in dent90115Dict.items():
    print(key)
    # check if 1383423, 1064909 and 1081787 are in the colId
    if not 1383423 in df[colId].values:
        print('1383423 not in the list')
    if not 1064909 in df[colId].values:
        print('1064909 not in the list')
    if not 1081787 in df[colId].values:
        print('1081787 not in the list')
    code = key.split('_')[0]
    # if not code.startswith('6'):
    #     print(f'Skipping {code}\n')
    #     continue
    df[colId] = df[colId].astype('Int64')
    mcColumns = findMCColumns(df, code)
    df = aggregator(df, mcColumns, colCE, colCEReason)
    countsDf = dfTemplate.copy()
    countsDf[colId] = df[colId]
    countsDf['Yes'] = df[mcColumns].apply(lambda x: (x == 1).sum() + (x=='1').sum(), axis=1)
    countsDf['No'] = df[mcColumns].apply(lambda x: (x == 0).sum() + (x=='0').sum(), axis=1)
    countsDf['Not Reviewed'] = df[mcColumns].apply(lambda x: (x == 'NA').sum(), axis=1)
    countsDf[labelBlank] = df[mcColumns].apply(lambda x: (x == '').sum() + x.isna().sum() + (x=='None').sum(), axis=1)
    countsDf['Total MC items'] = len(mcColumns)

    for q in rubricQues:
        countsDf[q] = df[q]
    for col in otherCols:
        countsDf[col] = df[col]

    countsDf['MC Score'] = (countsDf['Yes'] + notReviewedW*countsDf['Not Reviewed']) / (countsDf['Yes'] + countsDf['No'] + notReviewedW*countsDf['Not Reviewed'])
    countsDf['MC Score'] = countsDf['MC Score'].apply(lambda x: round(x, 2))
    

    
    countsDf[labelRubric] = countsDf.apply(lambda row: sum(row[ques] * rubricW[ques] / rubricDenom[ques] for ques in rubricQuesNoES), axis=1)/0.3
    countsDf[labelRubric] = countsDf[labelRubric].apply(lambda x: round(x, 2))
    
    countsDf['ES Weighted Score'] = countsDf['ES']/ rubricDenom['ES']
    countsDf['ES Weighted Score'] = countsDf['ES Weighted Score'].apply(lambda x: round(x, 2))
    
    countsDf['Total Score'] = (mcScoreW*countsDf['MC Score'] + rubricScoreW*countsDf[labelRubric])*100
    countsDf['Total Score'] = countsDf['Total Score'].apply(lambda x: round(x, 2))

    # Add 20% penalty if critical error is yes
    countsDf['Total Score CE Penalty (20%)'] = countsDf.apply(lambda row: row['Total Score'] * 0.8 if row[colCE] == 'Yes' else row['Total Score'], axis=1)
    countsDf['Total Score CE Penalty (20%)'] = countsDf['Total Score CE Penalty (20%)'].apply(lambda x: round(x, 2))

    # Add 10% penalty if critical error is yes
    countsDf['Total Score CE Penalty (10%)'] = countsDf.apply(lambda row: row['Total Score'] * 0.90 if row[colCE] == 'Yes' else row['Total Score'], axis=1)
    countsDf['Total Score CE Penalty (10%)'] = countsDf['Total Score CE Penalty (10%)'].apply(lambda x: round(x, 2))
    
    countsDf.sort_values('Total Score', ascending=False, inplace=True)

    # countsDf['Total Score'] = (mcScoreW*countsDf['MC Score'] + rubricScoreW*countsDf['Rubric Weighted Score'])*100
    # countsDf['Total Score'] = countsDf['Total Score'].apply(lambda x: round(x, 2))
    # get correlation matrix between MC and Rubric bar ES and ES
    countsDf['Item'] = key
    
    # display(countsDf.head())
    countsDfPaed = pd.concat([countsDfPaed, countsDf])
    # for i, row in df.iterrows():
    #     if row[colClinicType] == 'Paeds':
    #         print(f'Paeds row found: {row[colId]} {row[colDate]} {row[colSupervisor]}')

In [ ]:
display(countsDfPaed.head())
correlation_matrix = countsDfPaed[['MC Score', labelRubric, 'ES Weighted Score']].corr()
display(correlation_matrix)
correlation_matrix.to_csv(f'{folder}\\correlation_matrix.csv')
print(countsDfPaed[colId].value_counts())

In [ ]:
countsDfPaed.sort_values(colId, inplace=True)
countsDfPaed.to_excel(f'{folder}/{file} marks.xlsx', index=False)

In [ ]:
# Aggregate the data on Student ID, take average of Total Score
df = countsDfPaed.copy()
df[colId] = df[colId].astype('Int64')
aggFuncs = {col: 'first' for col in df.columns if col not in [colId, 'Item']}
aggFuncs.update({'Total Score': 'mean', 'Total Score CE Penalty (20%)': 'mean', 'Total Score CE Penalty (10%)': 'mean'})
df = df.groupby([colId], as_index=False).agg(aggFuncs)
df['Total Score'] = df['Total Score'].apply(lambda x: round(x, 2))
df['Total Score CE Penalty (20%)'] = df['Total Score CE Penalty (20%)'].apply(lambda x: round(x, 2))
df['Total Score CE Penalty (10%)'] = df['Total Score CE Penalty (10%)'].apply(lambda x: round(x, 2))
df = df[[colId, 'Total Score', colCE, colCEReason, 'Total Score CE Penalty (20%)', 'Total Score CE Penalty (10%)']]
df.sort_values('Total Score', ascending=False, inplace=True)
df.to_excel(f'{folder}/{file} marks aggregated.xlsx', index=False)

# Overall Subjects

In [ ]:
display(subjectDict.keys())
for key in subjectDict.keys():
    display(key)
    display(subjectDict[key].keys())
display(subjectDict['DENT90115']['013_LIMITED OE'])

In [ ]:
# dent90115 = subjectDict['DENT90115']
# display(dent90115.keys())
# # create a workbook with keys as sheets
# for key in dent90115.keys():
#     saveDf(dent90115[key], f'{folder}/DENT90115.xlsx', key)

# # merge all the sheets into one
# df = pd.DataFrame()
# for key in dent90115.keys():
#     df = pd.concat([df, dent90115[key]])

# saveDf(df, f'{folder}/DENT90115.xlsx', 'All')


#### Now we create report for each subject Now

In [ ]:
labelYes= 'Yes'
labelNo = 'No'
labelNA = 'Not Reviewed'
labelBlank = 'Left blank as part of form updates'
# colDate = 'RecordedDate'
with open(variableUtils.mcReferenceFile) as f:
    mcReferenceDict = json.load(f)

with open(variableUtils.othermcReferenceFile) as f:
    othermcReferenceDict = json.load(f)

otherMcRefdf = pd.DataFrame(list(othermcReferenceDict.items()), columns=['Abbreviation', 'Full Text'])

def custom_agg(series):
    unique_vals = series.dropna().unique()
    if len(unique_vals) == 0: # All values are NA
        return pd.NA
    if len(unique_vals) == 1: # Only one unique value 
        return unique_vals[0]
    else:
        if '1' in unique_vals:
            return '1'
        elif '0' in unique_vals:
            return '0'
        else:
            return pd.NA
        
def getCounts(df, col, dfCount, items):
    roleCounts = df[col].value_counts()
    for role, count in roleCounts.items():
        if role not in dfCount.columns:
            dfCount[role] = 0  # Add the new role as a column with zeros
        dfCount.at[items, role] = count
    # display(df[colPatient].value_counts())
    # display(df[colRole].value_counts())
    # display(dfCount)

def getValueCounts(df, targetColumn, colId):
    counts = df.groupby(colId)[targetColumn].value_counts().unstack(fill_value=0)
    counts.reset_index(inplace=True)
    # make colId the index
    counts.set_index(colId, inplace=True)
    return counts

# create one PDF for each subject
testSubject = 'ORAL20005'

def createDfs(testSubject, subjectDict, label = None): # label is either Simulation or Clinic
    """
    Create a DataFrame with the counts for each item in the subject
    """
    countsDfMCDict = {}
    countsDfStudDict = {}
    countsDfRoleDict = {}
    countsDfPatientDict = {}
    countsDfRKCDict = {}                              # for other checklist items
    itemReferenceDict = {}
    ageList = []
    
    itemCountsDf = pd.DataFrame(index = subjectDict[testSubject].keys(), columns = ['Yes', 'No', 'Not Reviewed', labelBlank]) # to store the counts for each item in the subject
    itemLenDf = pd.DataFrame(index = subjectDict[testSubject].keys(), columns = ['Length']) # to store the length of each item in the subject
    itemRoleDf = pd.DataFrame(index = subjectDict[testSubject].keys(), columns = []) # to store the role of each item in the subject
    itemPatientDf = pd.DataFrame(index = subjectDict[testSubject].keys(), columns = []) # to store the patient of each item in the subject
    itemCIDf = pd.DataFrame(columns = [colId, 'Date', 'Item', 'Reason']) # to store the critical incidents of each item in the subject
    
    for items in subjectDict[testSubject].keys():
        code = items.split('_')[0]
        tag = items.split('_')[1]
        tag = tag.replace('-', '/')
        # select items from mcReferenceDict with {code}_MC\d+ as key
        print(items, code, tag)
        respectiveMCKey = [key for key in mcReferenceDict.keys() if re.match(f'{code}_MC\d+', key)]
        respectiveMCKey = [key for key in respectiveMCKey if tag in key]
        # get the dict part of mcReferenceDict based on the respectiveMCKey
        mcRef = {key: mcReferenceDict[key] for key in respectiveMCKey}
        # print(mcRef)
        # Trun mcRef into a DataFrame
        if mcRef:
            mcRefdf = pd.DataFrame(list(mcRef.items()), columns=['Abbreviation', 'Full Text'])
        
            # extract MC\d+ from Abbreviation and replace it
            if code!= '114' and code != '115':
                mcRefdf['MC'] = mcRefdf['Abbreviation'].str.extract(r'(MC\d+)')
            else:
                mcRefdf['MC'] = mcRefdf['Abbreviation'].str.extract(r'(MC\d+)')
                extracted_values = mcRefdf['Abbreviation'].str.extract(r'(H/S|U/S)', expand=False)
                # print(extracted_values)
                # Add H/S or U/S to the MC column based on what is present in the Abbreviation
                mcRefdf['MC'] = mcRefdf['MC'] + ' ' + extracted_values
            # drop the Abbreviation column
            # print(mcRefdf.columns)
            # Swap columns
            mcRefdf = mcRefdf[['MC', 'Full Text']]
            # mcRefdf.drop(columns=['Abbreviation'], inplace=True)
            # display(mcRefdf)
        
        else:
            print(mcRef)
            mcRefdf = None
        itemReferenceDict[items] = mcRefdf

        # print(respectiveMCKey)
        df = subjectDict[testSubject][items]
        mcColumns = findMCColumns(df, code)        
        calcColumns = mcColumns + rubricQues
        # print(mcColumns)

        # display(items)
        
        # get highest value count from colClinicChoice
        if label is not None:
            countsClinic = df[colClinicChoice].value_counts()
            highestClinic = countsClinic.idxmax()
            # print(highestClinic, label)
            if label == 'Simulation' and label != highestClinic:
                print('Skipping', items)
                continue
            elif label == 'Clinic' and highestClinic == 'Simulation':
                print('Skipping', items)
                continue

        # print colId type
        # display(df[colId].dtype)
        df[colId] = df[colId].astype(int)
        # remove test  values like 111111 or 123456 from colId float dtype
        df = df[~df[colId].isin(invalidIDs)]
        # display(df.head())


        # Aggregate if tag is SIM (Take the best value for each column for each student)
        if tag == 'SIM':
            # Step 1: Define the aggregation functions for each group of columns
            aggFuncs = {col: 'first' for col in df.columns if col not in mcColumns + rubricQues}
            aggFuncs.update({col: custom_agg for col in mcColumns})
            aggFuncs.update({col: 'max' for col in rubricQues})
            df = df.groupby([colDate, colId], as_index=False).agg(aggFuncs)      # Step 2: Apply the combined aggregation functions


        # Get age distribution
        ageList += df[colAge].tolist()


        # Create a DataFrame to store the counts for each MC column
        countsDfMC = pd.DataFrame(index=mcColumns, columns=['Yes', 'No', 'Not Reviewed', labelBlank])
        for col in mcColumns:
            countsDfMC.at[col, 'Yes'] = (df[col] == 1).sum()
            countsDfMC.at[col, 'No'] = (df[col] == 0).sum()
            countsDfMC.at[col, 'Not Reviewed'] = (df[col] == 'NA').sum()
            countsDfMC.at[col, labelBlank] = (df[col] == '').sum() + df[col].isna().sum()
        
        # sort based on MC column
        countsDfMC['_sort_key'] = countsDfMC.index
        # extract digit
        countsDfMC['_sort_key'] = countsDfMC['_sort_key'].str.extract(r'MC(\d+)').astype(int)
        # display(countsDfMC)
        countsDfMC.sort_values(by='_sort_key', inplace=True)
        countsDfMC.drop(columns=['_sort_key'], inplace=True)
        countsDfMCDict[items] = countsDfMC
        # display(countsDfMC)

        # Create a DataFrame to store the counts for each student
        countsDfStu = pd.DataFrame(index=df.index, columns=['Yes', 'No', 'Not Reviewed', labelBlank])
        countsDfStu['Yes'] = df[mcColumns].apply(lambda x: (x == 1).sum() + (x=='1').sum(), axis=1)
        countsDfStu['No'] = df[mcColumns].apply(lambda x: (x == 0).sum() + (x=='0').sum(), axis=1)
        countsDfStu['Not Reviewed'] = df[mcColumns].apply(lambda x: (x == 'NA').sum(), axis=1)
        countsDfStu[labelBlank] = df[mcColumns].apply(lambda x: (x == '').sum() + x.isna().sum() + (x=='None').sum(), axis=1)
        countsDfStu[colId] = df[colId]
        countsDfStu['Yes/No'] = countsDfStu['Yes'] / (countsDfStu['Yes'] + countsDfStu['No'])
        countsDfStu = countsDfStu.groupby(colId).agg('sum')     # merge the rows on the same student ID
        countsDfStudDict[items] = countsDfStu

        # Create a DataFrame to store the counts for RKC, CC, ICC, PEC
        # remove the _supervisor from the column names
        df.columns = [re.sub(r'_supervisor', '', col) for col in df.columns]
        rkcColumns = findChecklistColumns(df, checklistMap)
        countsOtherChecklist = pd.DataFrame(index=rkcColumns, columns=['Yes', 'No', 'Not Reviewed', labelBlank])
        countsOtherChecklist['Yes'] = df[rkcColumns].apply(lambda x: (x == 1).sum(), axis=0)
        # display(df[rkcColumns].apply(lambda x: (x == 'Yes').sum(), axis=0))
        countsOtherChecklist['No'] = df[rkcColumns].apply(lambda x: (x == 0).sum(), axis=0)
        countsOtherChecklist['Not Reviewed'] = df[rkcColumns].apply(lambda x: (x == 'NA').sum(), axis=0)
        countsOtherChecklist[labelBlank] = df[rkcColumns].apply(lambda x: (x == '').sum() + x.isna().sum() + (x=='None').sum(), axis=0)
        countsDfRKCDict[items] = countsOtherChecklist
        # print(items)
        # remove rows with values in Yes, No and Not Reviewed as 0
        # countsOtherChecklist = countsOtherChecklist[(countsOtherChecklist['Yes'] != 0) | (countsOtherChecklist['No'] != 0) | (countsOtherChecklist['Not Reviewed'] != 0)]

        # print(rkcColumns)
        # display(countsOtherChecklist)
        
        # Get counts of values from colRole and colPatient for each student
        if tag != 'SIM':
            # Create a DataFrame to store the counts for each role for each student
            countsDfRole = getValueCounts(df, colRole, colId)
            countsDfRoleDict[items] = countsDfRole
            # display(countsDfRole.head())
            # display(countsDfRole[colId].value_counts())

            # Create a DataFrame to store the counts for each patient for each student
            countsDfPatient = getValueCounts(df, colPatient, colId)
            countsDfPatientDict[items] = countsDfPatient
            # display(countsDfPatient.head())
            # display(countsDfRole[colId].value_counts())

        # Add the sum of the counts to the itemCountsDf
        itemCountsDf.at[items, 'Yes'] = countsDfMC['Yes'].sum()
        itemCountsDf.at[items, 'No'] = countsDfMC['No'].sum()
        itemCountsDf.at[items, 'Not Reviewed'] = countsDfMC['Not Reviewed'].sum()
        itemCountsDf.at[items, labelBlank] = countsDfMC[labelBlank].sum()

        # Add the length of the item to the itemLenDf
        itemLenDf.at[items, 'Length'] = len(df)

        # Get counts of values from colPatient and colRole for entire item
        if tag != 'SIM':
            getCounts(df, colRole, itemRoleDf, items)
            # remove rows with total values of less than 2
            itemRoleDf = itemRoleDf[itemRoleDf.sum(axis=1) > 2]
            getCounts(df, colPatient, itemPatientDf, items)
        # display(itemRoleDf.head())
        # display(itemPatientDf.head())
        # break

        # Get the info for critical incidents
        criticalDf = df[df[colCE] == 'Yes']
        # Loop through each row in the filtered DataFrame
        rows = []
        for index, row in criticalDf.iterrows():
            # print(colId, row[colId])
            newRow = {
                colId: row[colId],
                'Date': row[colDate],
                'Item': items.replace('_', ' '),
                'Reason': row[colCEReason]
            }
            rows.append(newRow)
        itemCIDf = pd.concat([itemCIDf, pd.DataFrame(rows)], ignore_index=True)
    # display(itemCIDf)
    # collect items with 2or less rows
    removeItems = itemLenDf[itemLenDf['Length'] <= 2].index
    # itemLenDf = itemLenDf[itemLenDf['Length'] > 1]
    # itemCountsDf = itemCountsDf.drop(removeItems)
    # print(ageList)
    return countsDfMCDict, countsDfStudDict, countsDfRoleDict, \
    countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems 
# display(itemCountsDf)

    
countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf,\
itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems = createDfs(testSubject, subjectDict, 'Clinic')
print(ageList)
# display(itemRoleDf.head())
# display(itemPatientDf.head())
# display(itemLenDf)
# display(itemCountsDf)
# print(removeItems)

In [8]:
otherMcRefdf

,Abbreviation,Full Text
0,PEC MC1,Adjusts clinician chair correctly
1,PEC MC2,"Maintains good posture, positioning of self, p..."
2,ICC MC1,Maintains standards infection control througho...
3,ICC MC2,Handles and disposes of sharps in a safe manner
4,RKC MC1,"Maintains accurate, consistent, legible and co..."
5,CC MC1,Obtains informed consent from the parent/ guar...


### CODE FUNCTIONS

In [9]:
pageSize = (11.69 * 1* inch, 8.27 * 2 * inch) # page size
print(pageSize)
figsize = (pageSize[0] / 100, pageSize[1] / 100)
subplots_per_page = 2
# Define the margins
leftMargin = 0 * inch
rightMargin = 0 * inch
topMargin = 1 * inch
bottomMargin = 0 * inch

# Define the styles for the headings
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name='Center', alignment=1))  # Center alignment
headingStyle = ParagraphStyle('Heading1', parent=styles['Heading1'], fontSize=32, alignment=1)  # Centered
subheadingStyle = ParagraphStyle('Heading2', parent=styles['Heading2'], fontSize=24, alignment=1)  # Centered
subsubheadingStyle = ParagraphStyle('Heading3', parent=styles['Heading3'], fontSize=20, alignment=1)  # Centered
Checklistcolors = {'Yes': 'blue', 'No': 'orange', 'Not Reviewed': 'lightgrey', labelBlank: 'white'}
# Set the colorblind-friendly palette
sns.set_palette("colorblind")

# Function to create and return a plot image as BytesIO
def create_plot_image(fig):
        buf = BytesIO()
        fig.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        return buf

# Function to create a single subplot
def create_subplot(ax, subset, count=False, show_legend=False, colors = None, xticks = None):
    # drop labelBlank column if it exists
    if labelBlank in subset.columns:
        subset = subset.drop(columns = [labelBlank])
    # display(subset)
    if count:
        # display(subset)
        subset.plot(kind='bar', ax=ax, edgecolor='black', color='#87CEEB', legend=False)
    else:
        if colors is not None:
            subset.plot(kind='bar', stacked=True, ax=ax, edgecolor='black', color=colors, legend=show_legend)
        else:
            subset.plot(kind='bar', stacked=True, ax=ax, edgecolor='black', legend=show_legend)
        # Add the legend only if show_legend is True
        if show_legend:
            ax.legend(title='', loc='upper right', bbox_to_anchor=(1, 1), bbox_transform=ax.transAxes)
    ax.set_xlabel('')
    ax.set_ylabel('Count')
    desired_ticks = range(len(xticks))
    # print(desired_ticks)
    # ax.set_xticks(range(len(xticks)))
    # ax.set_xticklabels(xticks)
    # y ticks should be integers
    ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
    # Show gridlines on the y-axis
    ax.grid(axis='y', linestyle='--', alpha=0.6)
    # Convert index to strings to avoid errors with re.sub
    subset.index = [str(i) for i in subset.index]
    # x tick labels, remove _ and put space
    subset.index = [re.sub(r'_', ' ', i) for i in subset.index]
    ax.set_xticklabels(subset.index, rotation=90, ha='center')

# Function to add graphs to the document
def addGraphs(countsDf, elements, onlyCount=False, sortColumn=None, colors = None, startingSubplots = None):
    df = countsDf.copy()
    # Add a column for the total count
    df['Total'] = df.sum(axis=1)
    # Sort the DataFrame based on the total count in descending order
    if sortColumn is not None:
        df = df.sort_values(by=sortColumn, ascending=False)
    # Remove the rows with 0 total
    df = df[df['Total'] != 0]
    if not onlyCount and (df['Total'] == df[labelBlank]).all():
        elements.append(Paragraph("No data available yet", subsubheadingStyle))
        return 100
    # Drop the 'Total' column after sorting
    df = df.drop(columns=['Total', 'Yes/No'], errors='ignore')

    # Number of rows and columns for subplots
    max_bars_per_subplot = 12
    num_subplots = len(df) // max_bars_per_subplot + (1 if len(df) % max_bars_per_subplot > 0 else 0)
    # subplots_per_page = 2
    
    # Display starting subplots number of plots on the first page
    if startingSubplots is None:
        startingSubplots = subplots_per_page
        lastPageSubplots = num_subplots % subplots_per_page
    else:
        alreadyNumPlots = 3 - startingSubplots
        if alreadyNumPlots + num_subplots <= 3:
            lastPageSubplots = (alreadyNumPlots + num_subplots)
        else:
            lastPageSubplots = abs(num_subplots - startingSubplots) % subplots_per_page
        startingSubplots = min(startingSubplots, num_subplots)

    # figsize = (11.69, 15.54)  # A3 size in inches
    # get the suplots being plotted on the last page
    # lastPageSubplots = (num_subplots - startingSubplots) % subplots_per_page
    # Loop over the DataFrame and create subplots
    # for i in range(0, num_subplots, subplots_per_page):
    i = 0
    while i < num_subplots:
        if i == 0:
                current_subplots = startingSubplots
        else:
                current_subplots = subplots_per_page
        # print(i, current_subplots, (figsize[0], figsize[1] / subplots_per_page * current_subplots))
        fig, axes = plt.subplots(nrows=current_subplots, ncols=1, figsize=(figsize[0], figsize[1] / 3 * current_subplots))
        if current_subplots == 1:
            axes = [axes]
        else:
            axes = axes.flatten()
        
        for j in range(current_subplots):
            idx = i + j
            if idx < num_subplots:
                start_idx = idx * max_bars_per_subplot
                end_idx = start_idx + max_bars_per_subplot
                subset = df.iloc[start_idx:end_idx]
                xticks = subset.index
                # If there are fewer than max_bars_per_subplot, pad with empty rows
                if len(subset) < max_bars_per_subplot:
                    empty_rows = pd.DataFrame(0, index=[''] * (max_bars_per_subplot - len(subset)), columns=subset.columns)
                    subset = pd.concat([subset, empty_rows])
                # Show legend only for the first subplot
                create_subplot(axes[j], subset, onlyCount, show_legend=(i == 0 and j == 0), colors=colors, xticks=xticks)
            else:
                axes[j].remove()
        i+=current_subplots
        # fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1, 1), bbox_transform=fig.transFigure)
        plt.subplots_adjust(hspace=0.4)
        # plt.suptitle('Counts for each item')
        fig.tight_layout(rect=[0, 0, 1, 1])  # Adjust layout to make room for the legend
        plotImage = create_plot_image(fig)
        image = Image(plotImage)
        # print(image.drawWidth, image.drawHeight)
        
        # Resize image to fit within A3 page
        max_height = pageSize[1]  # Max height for A3 in points
        max_width = pageSize[0]  # Max width for A3 in points
        aspect_ratio = min(max_width / image.drawWidth, max_height / image.drawHeight)
        image.drawWidth *= aspect_ratio
        image.drawHeight *= aspect_ratio
        # print(image.drawWidth, image.drawHeight, aspect_ratio)
        elements.append(image)
        # if idx + 1 < num_subplots:
        #     elements.append(PageBreak())
        # elements.append(PageBreak())
        plt.close(fig)
    
    if lastPageSubplots == 0:
        lastPageSubplots = subplots_per_page
    return lastPageSubplots

def addLengthGraph(itemLenDf, elements):
    """
    Add a graph for the length of each item
    """
    subheading = Paragraph("Number of forms filled for each item", subheadingStyle)
    elements.append(subheading)
    elements.append(Spacer(1, 12))
    subplotcount = addGraphs(itemLenDf, elements, onlyCount=True, sortColumn='Length')
    if subplotcount > (subplots_per_page - 1):
        elements.append(PageBreak())
        firstpagesubplots = None
    else:
        firstpagesubplots = subplots_per_page - subplotcount
    print(f'After Length subplotcount: {subplotcount}')
    return firstpagesubplots

def addOverallMCGraph(itemCountsDf, elements, firstpagesubplots=None):
    subheading = Paragraph(" Marking Checklist Counts for each item", subheadingStyle)
    elements.append(subheading)
    elements.append(Spacer(1, 12))
    subplotcount = addGraphs(itemCountsDf, elements, onlyCount=False, sortColumn='Total', 
                             colors=Checklistcolors, startingSubplots=firstpagesubplots)
    if subplotcount > (subplots_per_page - 1):
        elements.append(PageBreak())
        firstpagesubplots = None
    else:
        firstpagesubplots = subplots_per_page - subplotcount
    print(f'After MC overall subplotcount: {subplotcount}')
    return firstpagesubplots    
    
def addAgeHistogram(ageList, elements):
    fig, ax = plt.subplots(figsize=(figsize[0], figsize[1]/2.2))  # A3 size in inches (landscape)
    # want 20 bins
    
    bins = list(range(0, 20, 2))  # Bins from 0 to 100 with step of 5
    # take max into account
    # bins = list(range(0, max(ageList) + 5, max((max(ageList)+5) // 20), 1))
    ax.hist(ageList, bins=bins, edgecolor='black')
    ax.set_title('')
    ax.set_xlabel('Age')
    ax.set_ylabel('Frequency')
    ax.set_xticks(list(bins))

    plt.subplots_adjust(hspace=0.4)
    fig.tight_layout(rect=[0, 0, 1, 1])  # Adjust layout to make room for the legend
    plotImage = create_plot_image(fig)
    image = Image(plotImage)
    
    # Resize image to fit within A3 page
    max_height = pageSize[1]  # Max height for A3 in points
    max_width = pageSize[0]  # Max width for A3 in points
    aspect_ratio = min(max_width / image.drawWidth, max_height / image.drawHeight)
    image.drawWidth *= aspect_ratio
    image.drawHeight *= aspect_ratio
    
    elements.append(image)
    plt.close(fig)


def addOverallRoleGraph(itemRoleDf, elements, firstpagesubplots=None):
    subheading = Paragraph("Role counts for each item", subheadingStyle)
    elements.append(subheading)
    elements.append(Spacer(1, 12))
    addGraphs(itemRoleDf, elements, False, 'Total', None)
    elements.append(PageBreak())

def addOverallPatientGraph(itemPatientDf, elements, firstpagesubplots=None):
    subheading = Paragraph("Patient counts for each item", subheadingStyle)
    elements.append(subheading)
    elements.append(Spacer(1, 12))
    addGraphs(itemPatientDf, elements, False, 'Total', None)
    elements.append(PageBreak())

def addMCReferenceTable(mcRefdf, elements, code= None):
    header = f"Marking Checklist Reference for {code}" if code is not None else "Marking Checklist Reference"
    elements.append(Paragraph(header, subheadingStyle))
    elements.append(Spacer(1, 12))
    if mcRefdf is None:
        elements.append(Paragraph("No marking checklist reference found", subsubheadingStyle))
        return
    largeFontStyle = ParagraphStyle('LargeFont', parent=styles['Normal'], fontSize=13, alignment=0)

    # Create a table with MC and Full Text columns
    data = [mcRefdf.columns.to_list()] + mcRefdf.values.tolist()
    for i in range(1, len(data)):
        data[i][1] = Paragraph(data[i][1], largeFontStyle)
        data[i][0] = Paragraph(data[i][0], largeFontStyle)
    table = Table(data, colWidths=[1.5 * inch, 9 * inch])

    # Set the style for the table text as large font
    

    table_style = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#9C27B0')),  # Header row
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor('#FFFFFF')),  # Header text
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center align all cells
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # Center align all cells
        ('GRID', (0, 0), (-1, -1), 1, colors.black),  # Add border around cells
        ('ALIGN', (3, 1), (3, -1), 'LEFT'),  # Left align Reason column cells
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold'),  # Change font to bold
        ('FONTSIZE', (0, 0), (-1, -1), 14),  # Increase font size
        ('BOTTOMPADDING', (0, 0), (-1, -1), 12),  # Increase bottom padding
        ('TOPPADDING', (0, 0), (-1, -1), 12),  # Increase top padding
    ])
    table.setStyle(table_style)
    elements.append(table)
    elements.append(PageBreak())



def addCriticalIncidents(itemCIDf, elements):
    styles = getSampleStyleSheet()
    # Ensure itemCIDf is not empty
    elements.append(Paragraph("Critical Incidents", subheadingStyle))
    elements.append(Spacer(1, 12))
    if itemCIDf.empty:
        elements.append(Paragraph("No critical incidents reported", subsubheadingStyle))
        return
    
    itemCICountDf = itemCIDf[colId].value_counts().reset_index()  # Group by the 'colId' column and count the occurrences
    itemCICountDf.columns = [colId, '# Critical Incidents']  # Rename the columns to 'student_id' and 'count'
    itemCICountDf.sort_values(by=colId, inplace=True)     # Sort the DataFrame by 'student_id'
    itemCICountDf['# Critical Incidents'] = itemCICountDf['# Critical Incidents'].astype(str)
    itemCICountDf[colId] = itemCICountDf[colId].astype(str)
    
    header = "Critical Errors Count"
    elements.append(Paragraph(header, subheadingStyle))
    elements.append(Spacer(1, 12))
    largeFontStyle1 = ParagraphStyle('LargeFont', parent=styles['Normal'], fontSize=13, alignment=1)

    # Create a table with MC and Full Text columns
    data = [itemCICountDf.columns.to_list()] + itemCICountDf.values.tolist()
    for i in range(1, len(data)):
        data[i][1] = Paragraph(data[i][1], largeFontStyle1)
        data[i][0] = Paragraph(data[i][0], largeFontStyle1)
    table = Table(data, colWidths=[1.5 * inch, 3 * inch])

    # Set the style for the table text as large font
    

    table_style = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#9C27B0')),  # Header row
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor('#FFFFFF')),  # Header text
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center align all cells
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # Center align all cells
        ('GRID', (0, 0), (-1, -1), 1, colors.black),  # Add border around cells
        ('ALIGN', (3, 1), (3, -1), 'LEFT'),  # Left align Reason column cells
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold'),  # Change font to bold
        ('FONTSIZE', (0, 0), (-1, -1), 14),  # Increase font size
        ('BOTTOMPADDING', (0, 0), (-1, -1), 12),  # Increase bottom padding
        ('TOPPADDING', (0, 0), (-1, -1), 12),  # Increase top padding
    ])
    table.setStyle(table_style)
    elements.append(table)
    # elements.append(PageBreak())
    elements.append(Spacer(1, 12))

    elements.append(Paragraph("Critical Incident Reasons", subheadingStyle))
    elements.append(Spacer(1, 12))
    
    itemCIDf['Date'] = pd.to_datetime(itemCIDf['Date'],format='mixed').dt.strftime('%d-%m-%Y')
    # Group by Student ID and Date
    # itemCIDf = itemCIDf.groupby([colId, colDate, 'Reason'])['Item'].apply(lambda x: ','.join(sorted(set(x)))).reset_index()
    # Rearrange the columns
    # itemCIDf = itemCIDf[[colId, colDate, 'Item', 'Reason']]

    # Sort by Item and then by Date
    itemCIDf = itemCIDf.sort_values(by=['Item', 'Date'])

    # Define a style for the text
    styles = getSampleStyleSheet()
    normalStyle = ParagraphStyle('Normal', parent=styles['Normal'], fontSize=12, alignment=0)
    largeFontStyle = ParagraphStyle('LargeFont', parent=styles['Normal'], fontSize=14, alignment=0)

    # Add the headers to the data
    data = [itemCIDf.columns.to_list()] + itemCIDf.applymap(str).values.tolist()

    # Wrap the text in the Reason column
    for i in range(1, len(data)):
        data[i][3] = Paragraph(data[i][3], largeFontStyle)
        data[i][2] = Paragraph(data[i][2], largeFontStyle)
    # Create the table with wider Reason column
    table = Table(data, colWidths=[1.2 * inch, 1.3 * inch, 2 * inch, 6 * inch])

    # Set the style for the table
    table_style = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#9C27B0')),  # Header row
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor('#FFFFFF')),  # Header text
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center align all cells
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # Center align all cells
        ('GRID', (0, 0), (-1, -1), 1, colors.black),  # Add border around cells
        ('ALIGN', (3, 1), (3, -1), 'LEFT'),  # Left align Reason column cells
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold'),  # Change font to bold
        ('FONTSIZE', (0, 0), (-1, -1), 14),  # Increase font size
        ('BOTTOMPADDING', (0, 0), (-1, -1), 12),  # Increase bottom padding
        ('TOPPADDING', (0, 0), (-1, -1), 12),  # Increase top padding
    ])

    # Apply alternating row colors based on 'Item'
    previous_item = None
    color = colors.whitesmoke
    for i in range(1, len(data)):
        current_item = data[i][2]
        if current_item != previous_item:
            color = colors.white if color == colors.lightskyblue else colors.lightskyblue
            previous_item = current_item
        table_style.add('BACKGROUND', (0, i), (-1, i), color)

    table.setStyle(table_style)

    # Add the table to the elements
    elements.append(table)
    elements.append(PageBreak())

def addItemWiseMCCounts(mcDf, elements, itemdisplay, mcRefdf):

    if mcDf.empty:
        return None
    elements.append(Paragraph(f"Counts for each MC column for {itemdisplay} ", subsubheadingStyle))
    elements.append(Spacer(1, 6))
    subplotcount = addGraphs(mcDf, elements, False, None, colors=Checklistcolors)
    if subplotcount > 99:
        elements.append(Spacer(1, 12))
        addMCReferenceTable(mcRefdf, elements, itemdisplay)
        # elements.append(PageBreak())
        return None
    print(f'After MC subplotcount: {subplotcount}')
    if subplotcount > (subplots_per_page - 1):
        elements.append(PageBreak())
        firstpagesubplots = None
    else:
        firstpagesubplots = subplots_per_page - subplotcount
    
    addMCReferenceTable(mcRefdf, elements, itemdisplay)
    return firstpagesubplots

def addItemWiseOtherChecklistCounts(mcDf, elements, itemdisplay, firstpagesubplots=None):

    if mcDf.empty:
        return None
    elements.append(Paragraph(f"Counts for other checklists for {itemdisplay} ", subsubheadingStyle))
    elements.append(Spacer(1, 6))
    subplotcount = addGraphs(mcDf, elements, False, 'Total', colors=Checklistcolors)
    print(f'After MC subplotcount: {subplotcount}')
    if subplotcount > (subplots_per_page - 1):
        elements.append(PageBreak())
        firstpagesubplots = None
    else:
        firstpagesubplots = 3 - subplotcount
    return firstpagesubplots

def addStudentWiseCounts(studDf, elements, itemdisplay, firstpagesubplots=None):
    if studDf.empty:
        return None
    elements.append(Paragraph(f"Counts for each student for {itemdisplay}", subsubheadingStyle))
    elements.append(Spacer(1, 6))
    subplotcount = addGraphs(studDf, elements, False, 'Yes/No', colors=Checklistcolors, startingSubplots=firstpagesubplots)
    print(f'After Stud subplotcount: {subplotcount}')
    if subplotcount > (subplots_per_page - 1):
        elements.append(PageBreak())
        firstpagesubplots = None
    else:
        firstpagesubplots = subplots_per_page - subplotcount
    return firstpagesubplots

def addStudentWiseRoleCounts(roleDf, elements, itemdisplay, firstpagesubplots=None):
    if roleDf.empty:
        return None
    elements.append(Paragraph(f"Role counts for each student for {itemdisplay}", subsubheadingStyle))
    elements.append(Spacer(1, 6))
    subplotcount = addGraphs(roleDf, elements, False, 'Total', None, startingSubplots=firstpagesubplots)
    print(f'After Role subplotcount: {subplotcount}')
    if subplotcount > (subplots_per_page - 1):
        elements.append(PageBreak())
        firstpagesubplots = None
    else:
        firstpagesubplots = subplots_per_page - subplotcount
    return firstpagesubplots

def addStudentWisePatientCounts(patientDf, elements, itemdisplay, firstpagesubplots=None):
    if patientDf.empty:
        return None
    elements.append(Paragraph(f"Patient counts for each student for {itemdisplay}", subsubheadingStyle))
    elements.append(Spacer(1, 6))
    subplotcount = addGraphs(patientDf, elements, False, 'Total', None, startingSubplots=firstpagesubplots)
    print(f'After Patient subplotcount: {subplotcount}')
    if subplotcount > (subplots_per_page - 1):
        elements.append(PageBreak())
        firstpagesubplots = None
    else:
        firstpagesubplots = subplots_per_page - subplotcount
    return firstpagesubplots

    # break
# createSubjectReport(folder, testSubject, itemLenDf, countsDfMCDict, countsDfStudDict, itemCountsDf)
# Example usage:
# Assuming testSubject, countsDfMCDict, countsDfStudDict, itemCountsDf are defined as per your data
# createSubjectReport(testSubject, countsDfMCDict, countsDfStudDict, itemCountsDf)


(841.68, 1190.8799999999999)


### GENERAL

In [10]:
def createSubjectReport(folder, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, removeItems):
    doc = SimpleDocTemplate(f"{folder}/{testSubject}.pdf", pagesize=pageSize, leftMargin=leftMargin,
    rightMargin=rightMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    elements = []

    # Add the title
    heading = Paragraph(f"{testSubject}", headingStyle)
    elements.append(heading)
    elements.append(Spacer(1, 12))
    
    # Add the length of each item
    addLengthGraph(itemLenDf, elements)
    
    # Add the MC counts for each item
    addOverallMCGraph(itemCountsDf, elements)
    
    # Add Role counts for each item
    addOverallRoleGraph(itemRoleDf, elements)

    # Add Patient counts for each item
    addOverallPatientGraph(itemPatientDf, elements)

    # Now add stuff for each item
    heading2 = Paragraph("Information for each item", headingStyle)
    elements.append(heading2)
    elements.append(Spacer(1, 12))
    items = countsDfMCDict.keys()
    
    for item in items:
        itemdisplay = item.replace('_', ' ')    
        elements.append(Paragraph(f"{itemdisplay}", subheadingStyle))
        elements.append(Spacer(1, 12))
        if item in removeItems:
            continue
        print(item)

         # Add graphs for Counts of MC values
        mcDf = countsDfMCDict[item]
        mcDf.index = [re.search(r'MC\d+', str(i)).group() if re.search(r'MC\d+', str(i)) else i for i in mcDf.index]
        firstpagesubplots = addItemWiseMCCounts(mcDf, elements, itemdisplay)

        # Add graphs for Counts Yes/No for each student
        studDf = countsDfStudDict[item] 
        firstpagesubplots = addStudentWiseCounts(studDf, elements, itemdisplay, firstpagesubplots)

        # Add the graphs for the role counts for each student
        if item in countsDfRoleDict.keys():
            roleDf = countsDfRoleDict[item]
            firstpagesubplots = addStudentWiseRoleCounts(roleDf, elements, itemdisplay, firstpagesubplots)

        # # Add the graphs for the patient counts for each student
        if item in countsDfPatientDict.keys():
            patientDf = countsDfPatientDict[item]
            firstpagesubplots = addStudentWisePatientCounts(patientDf, elements, itemdisplay, firstpagesubplots)
    
    elements.append(PageBreak())

    # Add the critical incidents
    addCriticalIncidents(itemCIDf, elements)

    doc.build(elements)


### ORAL10005

#### Simulation

In [11]:
def createSubjectReport10005(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, 
                             itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems):
    doc = SimpleDocTemplate(f"{folder}/{filename}.pdf", pagesize=pageSize, leftMargin=leftMargin,
    rightMargin=rightMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    elements = []

    # Add the title
    heading = Paragraph(f"{testSubject}", headingStyle)
    elements.append(heading)
    elements.append(Spacer(1, 12))
    
    # Add the length of each item
    firstpagesubplots = addLengthGraph(itemLenDf, elements)
    # elements.append(PageBreak())

    # print(ageList)
    # heading =  Paragraph("Patient's Age Distribution", subheadingStyle)
    # elements.append(heading)
    # elements.append(Spacer(1, 6))
    # addAgeHistogram(ageList, elements)
    # Add the MC counts for each item
    # firstpagesubplots = addOverallMCGraph(itemCountsDf, elements, firstpagesubplots)
    if firstpagesubplots is not None:
        elements.append(PageBreak())
    
    # Now add stuff for each item
    heading2 = Paragraph("Information for each item", headingStyle)
    elements.append(heading2)
    elements.append(Spacer(1, 12))
    items = countsDfMCDict.keys()
    print(removeItems)
    for item in items:
        code = item.split('_')[0]
        if item in removeItems:
            continue
        itemdisplay = item.replace('_', ' ')    
        elements.append(Paragraph(f"{itemdisplay}", subheadingStyle))
        elements.append(Spacer(1, 12))
        print(item)

         # Add graphs for Counts of MC values
        mcDf = countsDfMCDict[item]
        if code!= '114' and code!='115':
            mcDf.index = [re.search(r'MC\d+', str(i)).group() if re.search(r'MC\d+', str(i)) else i for i in mcDf.index]
        else:
            mcDf['newIndex'] = mcDf.index
            mcDf['newIndex'] = mcDf['newIndex'].str.extract(r'(MC\d+)')

            display(mcDf)
            extracted_values = mcDf.index.str.extract(r'(H/S|U/S)', expand=False)
                # print(extracted_values)
                # Add H/S or U/S to the MC column based on what is present in the Abbreviation
            mcDf['newIndex'] = mcDf['newIndex'] + ' ' + extracted_values
            mcDf.set_index('newIndex', inplace=True)
            
        firstpagesubplots = addItemWiseMCCounts(mcDf, elements, itemdisplay, itemReferenceDict[item])

        # Add graphs for Counts Yes/No for each student
        # studDf = countsDfStudDict[item] 
        # firstpagesubplots = addStudentWiseCounts(studDf, elements, itemdisplay, firstpagesubplots)

        # # Add the graphs for the role counts for each student
        # if item in countsDfRoleDict.keys():
        #     roleDf = countsDfRoleDict[item]
        #     firstpagesubplots = addStudentWiseRoleCounts(roleDf, elements, itemdisplay, firstpagesubplots)

        # # # Add the graphs for the patient counts for each student
        # if item in countsDfPatientDict.keys():
        #     patientDf = countsDfPatientDict[item]
        #     firstpagesubplots = addStudentWisePatientCounts(patientDf, elements, itemdisplay, firstpagesubplots)
        # if firstpagesubplots is not None:
        #     elements.append(PageBreak())

    # Add the critical incidents
    addCriticalIncidents(itemCIDf, elements)

    doc.build(elements)





In [12]:
testSubject = 'ORAL10005'
label = 'Simulation'
filename = f'{testSubject}_{label}'
print(testSubject)
countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems  = createDfs(testSubject, subjectDict, label)
createSubjectReport10005(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems)

ORAL10005
011_COE 011 COE
MC search pattern: 011_MC\d+
Skipping 011_COE
013_LIMITED OE 013 LIMITED OE
MC search pattern: 013_MC\d+
Skipping 013_LIMITED OE
022_I-O RAD 022 I/O RAD
MC search pattern: 022_MC\d+
Skipping 022_I-O RAD
071_DIAGNOSTIC MODEL 071 DIAGNOSTIC MODEL
MC search pattern: 071_MC\d+
111_CLINIC 111 CLINIC
MC search pattern: 111_MC\d+
Skipping 111_CLINIC
113_SIM 113 SIM
MC search pattern: 113_MC\d+
114_CLINIC 114 CLINIC
MC search pattern: 114_MC\d+
114_SIM 114 SIM
MC search pattern: 114_MC\d+
121_SIM REMIN 121 SIM REMIN
MC search pattern: 121_MC\d+
123_SIM 123 SIM
{}
MC search pattern: 123_MC\d+
131_CLINIC 131 CLINIC
MC search pattern: 131_MC\d+
Skipping 131_CLINIC
141_CLINIC 141 CLINIC
MC search pattern: 141_MC\d+
Skipping 141_CLINIC
161_CLINIC 161 CLINIC
MC search pattern: 161_MC\d+
161_SIM 161 SIM
MC search pattern: 161_MC\d+
221_CLINIC 221 CLINIC
MC search pattern: 221_MC\d+
Skipping 221_CLINIC
221_SIM 221 SIM
MC search pattern: 221_MC\d+
222_SIM 222 SIM
MC search pat

,Yes,No,Not Reviewed,Left blank as part of form updates,newIndex
114_MC1_supervisor (CLINIC U/S),0,0,49,8,MC1
114_MC2_supervisor (CLINIC U/S),1,0,48,8,MC2
114_MC3_supervisor (CLINIC U/S),2,0,47,8,MC3
114_MC4_supervisor (CLINIC U/S),2,0,47,8,MC4
114_MC5_supervisor (CLINIC U/S),2,0,47,8,MC5
114_MC6_supervisor (CLINIC U/S),1,0,48,8,MC6
114_MC7_supervisor (CLINIC U/S),1,0,48,8,MC7
114_MC8_supervisor (CLINIC U/S),2,0,47,8,MC8


After MC subplotcount: 1
114_SIM


,Yes,No,Not Reviewed,Left blank as part of form updates,newIndex
114_MC1_supervisor (SIM H/S),57,2,7,55,MC1
114_MC1_supervisor (SIM U/S),41,0,9,71,MC1
114_MC2_supervisor (SIM H/S),19,3,44,55,MC2
114_MC2_supervisor (SIM U/S),43,0,7,71,MC2
114_MC3_supervisor (SIM H/S),55,8,3,55,MC3
114_MC3_supervisor (SIM U/S),28,15,7,71,MC3
114_MC4_supervisor (SIM H/S),42,21,3,55,MC4
114_MC4_supervisor (SIM U/S),37,5,8,71,MC4
114_MC5_supervisor (SIM H/S),43,20,3,55,MC5
114_MC5_supervisor (SIM U/S),36,7,7,71,MC5


After MC subplotcount: 2
121_SIM REMIN
After MC subplotcount: 1
123_SIM
161_CLINIC
After MC subplotcount: 1
161_SIM
After MC subplotcount: 1
221_SIM
After MC subplotcount: 1
222_SIM
After MC subplotcount: 2
511_SIM
After MC subplotcount: 2
521_SIM
After MC subplotcount: 2
531_SIM
After MC subplotcount: 1
532_SIM
After MC subplotcount: 2
572_SIM
After MC subplotcount: 2


#### Clinic

In [13]:
testSubject = 'ORAL10005'
label = 'Clinic'
filename = f'{testSubject}_{label}'
countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems  = createDfs(testSubject, subjectDict, label)
createSubjectReport10005(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems)

011_COE 011 COE
MC search pattern: 011_MC\d+
013_LIMITED OE 013 LIMITED OE
MC search pattern: 013_MC\d+
022_I-O RAD 022 I/O RAD
MC search pattern: 022_MC\d+
071_DIAGNOSTIC MODEL 071 DIAGNOSTIC MODEL
MC search pattern: 071_MC\d+
Skipping 071_DIAGNOSTIC MODEL
111_CLINIC 111 CLINIC
MC search pattern: 111_MC\d+
113_SIM 113 SIM
MC search pattern: 113_MC\d+
Skipping 113_SIM
114_CLINIC 114 CLINIC
MC search pattern: 114_MC\d+
Skipping 114_CLINIC
114_SIM 114 SIM
MC search pattern: 114_MC\d+
Skipping 114_SIM
121_SIM REMIN 121 SIM REMIN
MC search pattern: 121_MC\d+
Skipping 121_SIM REMIN
123_SIM 123 SIM
{}
MC search pattern: 123_MC\d+
Skipping 123_SIM
131_CLINIC 131 CLINIC
MC search pattern: 131_MC\d+
141_CLINIC 141 CLINIC
MC search pattern: 141_MC\d+
161_CLINIC 161 CLINIC
MC search pattern: 161_MC\d+
Skipping 161_CLINIC
161_SIM 161 SIM
MC search pattern: 161_MC\d+
Skipping 161_SIM
221_CLINIC 221 CLINIC
MC search pattern: 221_MC\d+
221_SIM 221 SIM
MC search pattern: 221_MC\d+
Skipping 221_SIM
222

## ORAL20005

#### Simulation

In [14]:

testSubject = 'ORAL20005'
label = 'Simulation'
filename = f'{testSubject}_{label}'
print(testSubject)
countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems   = createDfs(testSubject, subjectDict, label)
createSubjectReport10005(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems )


ORAL20005
011_COE 011 COE
MC search pattern: 011_MC\d+
Skipping 011_COE
012_POE 012 POE
MC search pattern: 012_MC\d+
Skipping 012_POE
013_LIMITED OE 013 LIMITED OE
MC search pattern: 013_MC\d+
Skipping 013_LIMITED OE
014_CONSULTATION 014 CONSULTATION
MC search pattern: 014_MC\d+
Skipping 014_CONSULTATION
022_I-O RAD 022 I/O RAD
MC search pattern: 022_MC\d+
Skipping 022_I-O RAD
061_VITALITY 061 VITALITY
MC search pattern: 061_MC\d+
Skipping 061_VITALITY
072_PHOTOS 072 PHOTOS
MC search pattern: 072_MC\d+
Skipping 072_PHOTOS
111_CLINIC 111 CLINIC
MC search pattern: 111_MC\d+
Skipping 111_CLINIC
111_SIM 111 SIM
MC search pattern: 111_MC\d+
Skipping 111_SIM
113_CLINIC 113 CLINIC
MC search pattern: 113_MC\d+
Skipping 113_CLINIC
113_SIM 113 SIM
MC search pattern: 113_MC\d+
Skipping 113_SIM
114_CLINIC 114 CLINIC
MC search pattern: 114_MC\d+
Skipping 114_CLINIC
114_SIM 114 SIM
MC search pattern: 114_MC\d+
115_CLINIC 115 CLINIC
MC search pattern: 115_MC\d+
Skipping 115_CLINIC
121_CLINIC REMIN 12

,Yes,No,Not Reviewed,Left blank as part of form updates,newIndex
114_MC1_supervisor (SIM H/S),46,1,10,7,MC1
114_MC1_supervisor (SIM U/S),12,0,1,51,MC1
114_MC2_supervisor (SIM H/S),28,4,25,7,MC2
114_MC2_supervisor (SIM U/S),13,0,0,51,MC2
114_MC3_supervisor (SIM H/S),44,9,4,7,MC3
114_MC3_supervisor (SIM U/S),12,1,0,51,MC3
114_MC4_supervisor (SIM H/S),41,12,4,7,MC4
114_MC4_supervisor (SIM U/S),12,1,0,51,MC4
114_MC5_supervisor (SIM H/S),48,5,4,7,MC5
114_MC5_supervisor (SIM U/S),13,0,0,51,MC5


After MC subplotcount: 2
221_SIM
After MC subplotcount: 1
222_SIM
After MC subplotcount: 2
311_SIM
After MC subplotcount: 2
386_SIM
After MC subplotcount: 1
411_CLINIC
411_SIM
414_CLINIC CVEK
After MC subplotcount: 1
414_CLINIC DECIDUOUS
After MC subplotcount: 2
414_SIM CVEK
After MC subplotcount: 1
414_SIM DECIDUOUS
After MC subplotcount: 2
521_SIM
After MC subplotcount: 2
522_SIM
After MC subplotcount: 2
523_SIM
After MC subplotcount: 2
524_SIM
After MC subplotcount: 2
525_SIM
After MC subplotcount: 2
531_SIM
After MC subplotcount: 1
532_SIM
After MC subplotcount: 2
533_SIM
After MC subplotcount: 2
534_SIM
After MC subplotcount: 2
577_SIM
After MC subplotcount: 1
578_SIM
After MC subplotcount: 1
579_BONDING
After MC subplotcount: 1
586_SIM SEPARATORS
After MC subplotcount: 1
586_SIM TOOTH PREP
After MC subplotcount: 1
587_SIM SEPARATORS
After MC subplotcount: 1
587_SIM TOOTH PREP
After MC subplotcount: 1
656_SIM
After MC subplotcount: 1


#### Clinic

In [15]:
def createSubjectReport20005(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict,
                              countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems):
    doc = SimpleDocTemplate(f"{folder}/{filename}.pdf", pagesize=pageSize, leftMargin=leftMargin,
    rightMargin=rightMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    elements = []

    # Add the title
    heading = Paragraph(f"{testSubject}", headingStyle)
    elements.append(heading)
    elements.append(Spacer(1, 12))
    
    # Add the length of each item
    firstpagesubplots = addLengthGraph(itemLenDf, elements)
    
    heading =  Paragraph("Patient's Age Distribution", subheadingStyle)
    elements.append(heading)
    elements.append(Spacer(1, 6))
    ageDf = pd.read_excel('BOH2\CAF+v0.1_October152024_19.01\CAF v0.1_October 15, 2024_19.01 Paeds filtered.xlsx')
    ageList = ageDf[colAge].tolist()
    print(ageList)
    addAgeHistogram(ageList, elements)
    elements.append(PageBreak())
    # Add the MC counts for each item
    # firstpagesubplots = addOverallMCGraph(itemCountsDf, elements, firstpagesubplots)
    if firstpagesubplots is not None:
        elements.append(PageBreak())
    
    
    # Add Role counts for each item
    # addOverallRoleGraph(itemRoleDf, elements)
    # Now add stuff for each item
    heading2 = Paragraph("Information for each item", headingStyle)
    elements.append(heading2)
    elements.append(Spacer(1, 12))
    items = countsDfMCDict.keys()
    print(removeItems)
    for item in items:

        if item in removeItems:
            continue
        itemdisplay = item.replace('_', ' ')    
        elements.append(Paragraph(f"{itemdisplay}", subheadingStyle))
        elements.append(Spacer(1, 12))
        print(item)

        # Add graphs for Counts of MC values
        mcDf = countsDfMCDict[item]
        if code!= '114' and code!='115':
            mcDf.index = [re.search(r'MC\d+', str(i)).group() if re.search(r'MC\d+', str(i)) else i for i in mcDf.index]
        else:
            mcDf['newIndex'] = mcDf.index
            mcDf['newIndex'] = mcDf['newIndex'].str.extract(r'(MC\d+)')

            display(mcDf)
            extracted_values = mcDf.index.str.extract(r'(H/S|U/S)', expand=False)
                # print(extracted_values)
                # Add H/S or U/S to the MC column based on what is present in the Abbreviation
            mcDf['newIndex'] = mcDf['newIndex'] + ' ' + extracted_values
            mcDf.set_index('newIndex', inplace=True)
        firstpagesubplots = addItemWiseMCCounts(mcDf, elements, itemdisplay, itemReferenceDict[item])

        # Add graphs for other checklist counts
        rkcDf = countsDfRKCDict[item]
        firstpagesubplots = addItemWiseOtherChecklistCounts(rkcDf, elements, itemdisplay, firstpagesubplots)
        addMCReferenceTable(otherMcRefdf, elements)
        firstpagesubplots = None
        # Add graphs for Counts Yes/No for each student
        # studDf = countsDfStudDict[item] 
        # firstpagesubplots = addStudentWiseCounts(studDf, elements, itemdisplay, firstpagesubplots)

        # # Add the graphs for the role counts for each student
        # if item in countsDfRoleDict.keys():
        #     roleDf = countsDfRoleDict[item]
        #     firstpagesubplots = addStudentWiseRoleCounts(roleDf, elements, itemdisplay, firstpagesubplots)

        # # # Add the graphs for the patient counts for each student
        # if item in countsDfPatientDict.keys():
        #     patientDf = countsDfPatientDict[item]
        #     firstpagesubplots = addStudentWisePatientCounts(patientDf, elements, itemdisplay, firstpagesubplots)
        if firstpagesubplots is not None:
            elements.append(PageBreak())

    # Add the critical incidents
    addCriticalIncidents(itemCIDf, elements)

    doc.build(elements)






# DENT90115

In [16]:
def createSubjectReportSim(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, 
                             itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems):
    doc = SimpleDocTemplate(f"{folder}/{filename}.pdf", pagesize=pageSize, leftMargin=leftMargin,
    rightMargin=rightMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    elements = []

    # Add the title
    heading = Paragraph(f"{testSubject}", headingStyle)
    elements.append(heading)
    elements.append(Spacer(1, 12))
    
    # Add the length of each item
    firstpagesubplots = addLengthGraph(itemLenDf, elements)
    # elements.append(PageBreak())

    # print(ageList)
    # heading =  Paragraph("Patient's Age Distribution", subheadingStyle)
    # elements.append(heading)
    # elements.append(Spacer(1, 6))
    # addAgeHistogram(ageList, elements)
    # Add the MC counts for each item
    # firstpagesubplots = addOverallMCGraph(itemCountsDf, elements, firstpagesubplots)
    if firstpagesubplots is not None:
        elements.append(PageBreak())
    
    # Now add stuff for each item
    heading2 = Paragraph("Information for each item", headingStyle)
    elements.append(heading2)
    elements.append(Spacer(1, 12))
    items = countsDfMCDict.keys()
    print(removeItems)
    for item in items:
        code = item.split('_')[0]
        if item in removeItems:
            continue
        itemdisplay = item.replace('_', ' ')    
        elements.append(Paragraph(f"{itemdisplay}", subheadingStyle))
        elements.append(Spacer(1, 12))
        print(item)

         # Add graphs for Counts of MC values
        mcDf = countsDfMCDict[item]
        if code!= '114' and code!='115':
            mcDf.index = [re.search(r'MC\d+', str(i)).group() if re.search(r'MC\d+', str(i)) else i for i in mcDf.index]
        else:
            mcDf['newIndex'] = mcDf.index
            mcDf['newIndex'] = mcDf['newIndex'].str.extract(r'(MC\d+)')

            display(mcDf)
            extracted_values = mcDf.index.str.extract(r'(H/S|U/S)', expand=False)
                # print(extracted_values)
                # Add H/S or U/S to the MC column based on what is present in the Abbreviation
            mcDf['newIndex'] = mcDf['newIndex'] + ' ' + extracted_values
            mcDf.set_index('newIndex', inplace=True)
            
        firstpagesubplots = addItemWiseMCCounts(mcDf, elements, itemdisplay, itemReferenceDict[item])

        # Add graphs for Counts Yes/No for each student
        # studDf = countsDfStudDict[item] 
        # firstpagesubplots = addStudentWiseCounts(studDf, elements, itemdisplay, firstpagesubplots)

        # # Add the graphs for the role counts for each student
        # if item in countsDfRoleDict.keys():
        #     roleDf = countsDfRoleDict[item]
        #     firstpagesubplots = addStudentWiseRoleCounts(roleDf, elements, itemdisplay, firstpagesubplots)

        # # # Add the graphs for the patient counts for each student
        # if item in countsDfPatientDict.keys():
        #     patientDf = countsDfPatientDict[item]
        #     firstpagesubplots = addStudentWisePatientCounts(patientDf, elements, itemdisplay, firstpagesubplots)
        # if firstpagesubplots is not None:
        #     elements.append(PageBreak())

    # Add the critical incidents
    addCriticalIncidents(itemCIDf, elements)

    doc.build(elements)





In [17]:
testSubject = 'DENT90115'
label = 'Simulation'
filename = f'{testSubject}_{label}'
print(testSubject)
countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems  = createDfs(testSubject, subjectDict, label)
display(countsDfMCDict['013_LIMITED OE'])
createSubjectReport10005(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems)

DENT90115
013_LIMITED OE 013 LIMITED OE
MC search pattern: 013_MC\d+
113_CLINIC 113 CLINIC
MC search pattern: 113_MC\d+
121_SIM REMIN 121 SIM REMIN
MC search pattern: 121_MC\d+
161_SIM 161 SIM
MC search pattern: 161_MC\d+
386_SIM 386 SIM
MC search pattern: 386_MC\d+
411_SIM 411 SIM
MC search pattern: 411_MC\d+
414_SIM CVEK 414 SIM CVEK
MC search pattern: 414_MC\d+
414_SIM DECIDUOUS 414 SIM DECIDUOUS
MC search pattern: 414_MC\d+
415_SIM ACCESS 415 SIM ACCESS
MC search pattern: 415_MC\d+
415_SIM WORK LENGTH 415 SIM WORK LENGTH
MC search pattern: 415_MC\d+
416_SIM RCT ADD 416 SIM RCT ADD
MC search pattern: 416_MC\d+
417_SIM CONE FIT 417 SIM CONE FIT
MC search pattern: 417_MC\d+
417_SIM ROOT FLG 417 SIM ROOT FLG
MC search pattern: 417_MC\d+
418_SIM CONE FIT 418 SIM CONE FIT
MC search pattern: 418_MC\d+
418_SIM ROOT FLG 418 SIM ROOT FLG
MC search pattern: 418_MC\d+
513_SIM 513 SIM
MC search pattern: 513_MC\d+
514_SIM 514 SIM
MC search pattern: 514_MC\d+
521_SIM 521 SIM
MC search pattern: 52

,Yes,No,Not Reviewed,Left blank as part of form updates
013_MC1,0,0,0,99
013_MC2,0,0,0,99
013_MC3,0,0,0,99
013_MC4,0,0,0,99
013_MC5,0,0,0,99
013_MC6,0,0,0,99
013_MC7,0,0,0,99
013_MC8,0,0,0,99


After Length subplotcount: 2
Index(['411_SIM', '521_SIM', '533_SIM', '555_SIM', '572_SIM',
       '586_SIM SEPARATORS', '587_SIM TOOTH PREP'],
      dtype='object')
013_LIMITED OE
113_CLINIC
121_SIM REMIN
After MC subplotcount: 1
161_SIM
After MC subplotcount: 1
386_SIM
After MC subplotcount: 1
414_SIM CVEK
After MC subplotcount: 1
414_SIM DECIDUOUS
After MC subplotcount: 2
415_SIM ACCESS
After MC subplotcount: 1
415_SIM WORK LENGTH
After MC subplotcount: 1
416_SIM RCT ADD
After MC subplotcount: 1
417_SIM CONE FIT
After MC subplotcount: 1
417_SIM ROOT FLG
After MC subplotcount: 1
418_SIM CONE FIT
After MC subplotcount: 2
418_SIM ROOT FLG
After MC subplotcount: 1
513_SIM
After MC subplotcount: 2
514_SIM
After MC subplotcount: 2
522_SIM
After MC subplotcount: 2
523_SIM
After MC subplotcount: 2
524_SIM
After MC subplotcount: 2
525_SIM
After MC subplotcount: 2
531_SIM
After MC subplotcount: 1
532_SIM
After MC subplotcount: 2
534_SIM
After MC subplotcount: 2
535_SIM
After MC subplotcount: 2

In [ ]:
# save dent90115 dict as workbook with keys as sheet names
for sheetname, df in dent90115Dict.items():
    saveDf(dent90115Dict[sheetname], f'{folder}/DENT90115.xlsx', sheetname)

In [ ]:
fixedprosdict = {}
endodict = {}
paedsdict = {}
# fixed pros items are of type 6XX
for sheetname, df in dent90115Dict.items():
    if sheetname.startswith('6'):
        fixedprosdict[sheetname] = df
    elif sheetname.startswith('4'):
        endodict[sheetname] = df
    elif sheetname.startswith('5'):
        paedsdict[sheetname] = df

overallDict = {'Fixed Prosthodontics': fixedprosdict, 'Endodontics': endodict, 'Paediatric': paedsdict}

In [ ]:
testSubject = 'Fixed Prosthodontics'
label = 'Simulation'
filename = f'DENT90115_{testSubject}_{label}'
print(testSubject)
countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems  = createDfs(testSubject, overallDict, label)
createSubjectReport10005(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems)

In [ ]:
testSubject = 'Endodontics'
label = 'Simulation'
filename = f'DENT90115_{testSubject}_{label}'
print(testSubject)
countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems  = createDfs(testSubject, overallDict, label)
createSubjectReport10005(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems)

In [ ]:
testSubject = 'Paediatric'
label = 'Simulation'
filename = f'DENT90115_{testSubject}_{label}'
print(testSubject)
countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems  = createDfs(testSubject, overallDict, label)
createSubjectReport10005(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems)

In [18]:
testSubject = 'ORAL20005'
label = 'Clinic'
filename = f'{testSubject}_{label}'
print(testSubject)
countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems  = createDfs(testSubject, subjectDict, label)
createSubjectReport20005(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems)

ORAL20005
011_COE 011 COE
MC search pattern: 011_MC\d+
012_POE 012 POE
MC search pattern: 012_MC\d+
013_LIMITED OE 013 LIMITED OE
MC search pattern: 013_MC\d+
014_CONSULTATION 014 CONSULTATION
MC search pattern: 014_MC\d+
022_I-O RAD 022 I/O RAD
MC search pattern: 022_MC\d+
061_VITALITY 061 VITALITY
MC search pattern: 061_MC\d+
072_PHOTOS 072 PHOTOS
MC search pattern: 072_MC\d+
111_CLINIC 111 CLINIC
MC search pattern: 111_MC\d+
111_SIM 111 SIM
MC search pattern: 111_MC\d+
113_CLINIC 113 CLINIC
MC search pattern: 113_MC\d+
113_SIM 113 SIM
MC search pattern: 113_MC\d+
114_CLINIC 114 CLINIC
MC search pattern: 114_MC\d+
114_SIM 114 SIM
MC search pattern: 114_MC\d+
Skipping 114_SIM
115_CLINIC 115 CLINIC
MC search pattern: 115_MC\d+
121_CLINIC REMIN 121 CLINIC REMIN
MC search pattern: 121_MC\d+
121_SIM REMIN 121 SIM REMIN
MC search pattern: 121_MC\d+
123_CLINIC 123 CLINIC
MC search pattern: 123_MC\d+
123_SIM 123 SIM
{}
MC search pattern: 123_MC\d+
131_CLINIC 131 CLINIC
MC search pattern: 131

### ORAL30002


#### Clinic

In [19]:
testSubject = 'ORAL30002'
label = 'Clinic'
filename = f'{testSubject}_{label}'
print(testSubject)
countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems  = createDfs(testSubject, subjectDict, label)
createSubjectReport20005(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems)


ORAL30002
011_COE 011 COE
MC search pattern: 011_MC\d+
012_POE 012 POE
MC search pattern: 012_MC\d+
013_LIMITED OE 013 LIMITED OE
MC search pattern: 013_MC\d+
014_CONSULTATION 014 CONSULTATION
MC search pattern: 014_MC\d+
022_I-O RAD 022 I/O RAD
MC search pattern: 022_MC\d+
061_VITALITY 061 VITALITY
MC search pattern: 061_MC\d+
072_PHOTOS 072 PHOTOS
MC search pattern: 072_MC\d+
111_CLINIC 111 CLINIC
MC search pattern: 111_MC\d+
113_CLINIC 113 CLINIC
MC search pattern: 113_MC\d+
114_CLINIC 114 CLINIC
MC search pattern: 114_MC\d+
115_CLINIC 115 CLINIC
MC search pattern: 115_MC\d+
121_CLINIC REMIN 121 CLINIC REMIN
MC search pattern: 121_MC\d+
123_CLINIC 123 CLINIC
MC search pattern: 123_MC\d+
131_CLINIC 131 CLINIC
MC search pattern: 131_MC\d+
141_CLINIC 141 CLINIC
MC search pattern: 141_MC\d+
142_CLINIC 142 CLINIC
MC search pattern: 142_MC\d+
161_CLINIC 161 CLINIC
MC search pattern: 161_MC\d+
221_CLINIC 221 CLINIC
MC search pattern: 221_MC\d+
222_CLINIC 222 CLINIC
MC search pattern: 222_M

### DENT90120

#### Clinic

In [20]:
testSubject = 'DENT90120'
label = 'Clinic'
filename = f'{testSubject}_{label}'
print(testSubject)
countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems  = createDfs(testSubject, subjectDict, label)
createSubjectReport20005(folder, filename, testSubject, countsDfMCDict, countsDfStudDict, countsDfRoleDict, countsDfPatientDict, itemCountsDf, itemLenDf, itemRoleDf, itemPatientDf, itemCIDf, countsDfRKCDict, itemReferenceDict, ageList, removeItems)

DENT90120
011_COE 011 COE
MC search pattern: 011_MC\d+
012_POE 012 POE
MC search pattern: 012_MC\d+
013_LIMITED OE 013 LIMITED OE
MC search pattern: 013_MC\d+
014_CONSULTATION 014 CONSULTATION
MC search pattern: 014_MC\d+
022_I-O RAD 022 I/O RAD
MC search pattern: 022_MC\d+
061_VITALITY 061 VITALITY
MC search pattern: 061_MC\d+
071_DIAGNOSTIC MODEL 071 DIAGNOSTIC MODEL
MC search pattern: 071_MC\d+
072_PHOTOS 072 PHOTOS
MC search pattern: 072_MC\d+
074_PHYSICAL MODELS 074 PHYSICAL MODELS
MC search pattern: 074_MC\d+
111_CLINIC 111 CLINIC
MC search pattern: 111_MC\d+
113_CLINIC 113 CLINIC
MC search pattern: 113_MC\d+
114_CLINIC 114 CLINIC
MC search pattern: 114_MC\d+
115_CLINIC 115 CLINIC
MC search pattern: 115_MC\d+
121_CLINIC REMIN 121 CLINIC REMIN
MC search pattern: 121_MC\d+
123_CLINIC 123 CLINIC
MC search pattern: 123_MC\d+
131_CLINIC 131 CLINIC
MC search pattern: 131_MC\d+
141_CLINIC 141 CLINIC
MC search pattern: 141_MC\d+
142_CLINIC 142 CLINIC
MC search pattern: 142_MC\d+
161_CLIN